Neste projeto prático, o objetivo é implementar o algoritmo de treinamento mediante Aprendizado Supervisionado 
do neurônio Perceptron de Rosenblatt aplicado em problemas de classificação. 

Equipe:
1. Afonso Ramos Garcia Neto | ifonso.developer@gmail.com | iFonso - 2215080047
2. Erik Gustavo Lima de Oliveira | erik.exatas10@gmail.com | ErikExatas - 2115080049
3. David Augusto De Oliveira E Silva | david-augusto-silva - 2115080006
4. Lilian Iazzai De Souza Oliveira | lilianiazzai - 2215080018
5. Vitor Nascimento Aguiar | Vtaguiar1909 - 2115080055

In [6]:
# identificador do arquivo
id = (7+9+6+8+5)%4
print(id)

3


In [5]:
# bibliotecas necessárias
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import sklearn